In [1]:
from ipyparallel import Client

In [2]:
c = None
def setup_client(client_arg = None):
    global c
    if client_arg:
        c = Client(client_arg)
    else:
        c = Client()
    print('Nr of Engines:', len(c))

In [3]:
setup_client("/home/shiroten/.ipython/profile_default/security/ipcontroller-client.json")

Nr of Engines: 8


In [4]:
%px %load_ext cython

[stdout:0] 
The cython extension is already loaded. To reload it, use:
  %reload_ext cython
[stdout:1] 
The cython extension is already loaded. To reload it, use:
  %reload_ext cython
[stdout:2] 
The cython extension is already loaded. To reload it, use:
  %reload_ext cython
[stdout:3] 
The cython extension is already loaded. To reload it, use:
  %reload_ext cython
[stdout:4] 
The cython extension is already loaded. To reload it, use:
  %reload_ext cython
[stdout:5] 
The cython extension is already loaded. To reload it, use:
  %reload_ext cython
[stdout:6] 
The cython extension is already loaded. To reload it, use:
  %reload_ext cython
[stdout:7] 
The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [5]:
%%px
import os
os.mkdir('./pubyc_renderer_refactor')

CompositeError: one or more exceptions from call to method: execute
[0:execute]: FileExistsError: [Errno 17] Die Datei existiert bereits: './pubyc_renderer_refactor'
[1:execute]: FileExistsError: [Errno 17] Die Datei existiert bereits: './pubyc_renderer_refactor'
[2:execute]: FileExistsError: [Errno 17] Die Datei existiert bereits: './pubyc_renderer_refactor'
[3:execute]: FileExistsError: [Errno 17] Die Datei existiert bereits: './pubyc_renderer_refactor'
.... 4 more exceptions ...

In [6]:
%%px 
%%file ./pubyc_renderer_refactor/calculation_function.pxd
cdef void calculate_distance (double [:, :], double[:, :], int, int, int) nogil
cdef void calculate_magnitude (double [:, :], double[:], int) nogil
cdef void calculate_force (double [:], double [:, :],  double [:], int, int, double[:, :], int) nogil
cdef void calculate_acceleration (double [:, :], double [:], int, double[:, :], int) nogil
cdef void update_position (double [:, :], double [:, :], double [:, :],double [:, :], double, int, int) nogil
cdef void update_speed (double [:, :], double [:, :], double [:, :], double, int, int) nogil
cdef double [:] single_planet(double, double [:, :], double [:, :], double [:, :]
                              , double [:, :], double [:], int, int, int)

[stdout:0] Overwriting ./pubyc_renderer_refactor/calculation_function.pxd
[stdout:1] Overwriting ./pubyc_renderer_refactor/calculation_function.pxd
[stdout:2] Overwriting ./pubyc_renderer_refactor/calculation_function.pxd
[stdout:3] Overwriting ./pubyc_renderer_refactor/calculation_function.pxd
[stdout:4] Overwriting ./pubyc_renderer_refactor/calculation_function.pxd
[stdout:5] Overwriting ./pubyc_renderer_refactor/calculation_function.pxd
[stdout:6] Overwriting ./pubyc_renderer_refactor/calculation_function.pxd
[stdout:7] Overwriting ./pubyc_renderer_refactor/calculation_function.pxd


In [7]:
%%px 
%%file ./pubyc_renderer_refactor/calculation_function.pyx
cimport cython
#Abstands Funktion
#Abstand_Sonne = position[1] - position[0]

@cython.boundscheck(False)
@cython.cdivision(True)
cdef void calculate_distance (double [:, :] position_view , double[:, :] result ,int head, int tail, int id) nogil: 
    result[id][0] = position_view[head][0] - position_view[tail][0]
    result[id][1] = position_view[head][1] - position_view[tail][1]
    result[id][2] = position_view[head][2] - position_view[tail][2]
    
from libc.math cimport sqrt
#Betrags Funktion
#Betrag_Erde  = sqrt( Abstand_Erde**2 + Abstand_Erde**2 + Abstand_Erde**2)
@cython.boundscheck(False)
@cython.cdivision(True)
cdef void calculate_magnitude (double [:, :] abstand_view , double[:] result, int id) nogil:
    result[id] = sqrt(abstand_view[id][0]**2 + abstand_view[id][1]**2 + abstand_view[id][2]**2)
    
#Kraft Funktion
#Kraft_Erde = G * (masse_erde * masse_sonne / (Betrag_Erde  ** 3)) * Abstand_Erde
@cython.boundscheck(False)
@cython.cdivision(True)
cdef void calculate_force (double [:] masse_view, double [:, :] abstand_view,  double [:] Betrag, \
                  int head, int tail, double[:, :] result, int id) nogil:
    
    cdef double g1 = 6.672
    cdef double g2 = 10
    cdef double g3 = -11
    
    cdef double G = g1 * g2 ** g3
    cdef double betrag = Betrag[id] ** 3

    cdef double masse_multyply = masse_view[head] * masse_view[tail]
    
    result[id][0] = G * (masse_multyply / betrag) * abstand_view[id][0]
    result[id][1] = G * (masse_multyply / betrag) * abstand_view[id][1]
    result[id][2] = G * (masse_multyply / betrag) * abstand_view[id][2]
    
#Beschleunigungs Funktion
#Beschleunigung_Erde  = Kraft_Erde  / masse
@cython.boundscheck(False)
@cython.cdivision(True)
cdef void calculate_acceleration (double [:, :] kraft_view , double [:] masse_view, \
                          int massen_index, double[:, :] result, int id) nogil:
    cdef float G = 6.672 * 10 ** -11
    result[id][0] = kraft_view[id][0]  / masse_view[massen_index]
    result[id][1] = kraft_view[id][1]  / masse_view[massen_index]
    result[id][2] = kraft_view[id][2]  / masse_view[massen_index]
    
#Positions Aktualliersierung
#position = position + dt * geschwindigkeit  + ((dt ** 2) / 2) * Beschleunigung_Erde
@cython.boundscheck(False)
@cython.cdivision(True)
cdef void update_position (double [:, :] position_view , double [:, :] new_postion_view, \
                   double [:, :] speed_view, double [:, :] beschleunigung_view, \
                        double dt, int planet_index, int id) nogil:
    
    new_postion_view[planet_index][0] = position_view[planet_index][0]  + dt * speed_view[planet_index][0] \
                                                + ((dt ** 2) / 2) * beschleunigung_view[id][0]
        
    new_postion_view[planet_index][1] = position_view[planet_index][1]  + dt * speed_view[planet_index][1] \
                                                + ((dt ** 2) / 2) * beschleunigung_view[id][1]
        
    new_postion_view[planet_index][2] = position_view[planet_index][2]  + dt * speed_view[planet_index][2] \
                                                + ((dt ** 2) / 2) * beschleunigung_view[id][2]
        
#Geschwindigkeits Aktuallisierung
#geschwindigkeit[0][0]  += dt * Beschleunigung_Erde[0]
@cython.boundscheck(False)
@cython.cdivision(True)
cdef void update_speed (double [:, :] speed_view, double [:, :] new_speed_view, \
                        double [:, :] beschleunigung_view, double dt ,int planet_index, int id) nogil:
    new_speed_view[planet_index][0] = speed_view[planet_index][0] + dt * beschleunigung_view[id][0]
    new_speed_view[planet_index][1] = speed_view[planet_index][1] + dt * beschleunigung_view[id][1]
    new_speed_view[planet_index][2] = speed_view[planet_index][2] + dt * beschleunigung_view[id][2]
        
from cython.parallel import prange , threadid
from multiprocessing import cpu_count
import numpy as np    
@cython.boundscheck(False)
@cython.cdivision(True)       
cpdef double [:] single_planet(double args_dt, double [:, :] position_view, double [:, :] new_position_view, 
                        double [:, :] speed_view, double [:, :] new_speed_view, 
                        double [:] masse_view, int current_planet, int thread_num, int chunksize):
    #Statische Variable
    cdef double dt = args_dt
    cdef int number_planets = len(position_view)
    cdef int i, planet
    cdef int id
    
    #Temporaere MemoryView zum ZwischenSpeichern
    cdef double [:, :] distance = np.zeros((thread_num, 3),  dtype=np.float64)
    cdef double [:]    magnitude = np.zeros(thread_num,  dtype=np.float64)
    cdef double [:, :] single_force = np.zeros((thread_num, 3),  dtype=np.float64)
    cdef double [:, :] all_force = np.zeros((thread_num, 3),  dtype=np.float64)
    cdef double [:, :] acceleration = np.zeros((thread_num, 3),  dtype=np.float64)
    cdef double [:]    return_value = np.zeros(6,  dtype=np.float64)
    
    id = threadid()
    all_force[id][0] = 0
    all_force[id][1] = 0
    all_force[id][2] = 0

    for planet in prange(number_planets, nogil=True, 
                        schedule='runtime'):  

        if (planet != current_planet):
            calculate_distance(position_view, distance, planet, current_planet, id)
            calculate_magnitude(distance, magnitude, id)
            calculate_force(masse_view, distance, magnitude, planet, current_planet, single_force, id)
            all_force[id][0] += single_force[id][0]
            all_force[id][1] += single_force[id][1]
            all_force[id][2] += single_force[id][2]

    calculate_acceleration(all_force, masse_view, current_planet, acceleration, id)
    update_position(position_view, new_position_view, speed_view, acceleration, dt, current_planet, id) 
    update_speed(speed_view , new_speed_view, acceleration, dt, current_planet, id)     
    
    return_value[0] = position_view[current_planet][0]
    return_value[1] = position_view[current_planet][1]
    return_value[2] = position_view[current_planet][2]
    return_value[3] = speed_view[current_planet][3]
    return_value[4] = speed_view[current_planet][4]
    return_value[5] = speed_view[current_planet][5]
                                               
    return return_value 

[stdout:0] Overwriting ./pubyc_renderer_refactor/calculation_function.pyx
[stdout:1] Overwriting ./pubyc_renderer_refactor/calculation_function.pyx
[stdout:2] Overwriting ./pubyc_renderer_refactor/calculation_function.pyx
[stdout:3] Overwriting ./pubyc_renderer_refactor/calculation_function.pyx
[stdout:4] Overwriting ./pubyc_renderer_refactor/calculation_function.pyx
[stdout:5] Overwriting ./pubyc_renderer_refactor/calculation_function.pyx
[stdout:6] Overwriting ./pubyc_renderer_refactor/calculation_function.pyx
[stdout:7] Overwriting ./pubyc_renderer_refactor/calculation_function.pyx


In [8]:
%%px 
%%file ./pubyc_renderer_refactor/simulation_engine.pyx
from ipyparallel import Client
from cython.parallel import prange , threadid
from multiprocessing import cpu_count
from calculation_function import single_planet
import numpy as np
cimport cython 

def single_step_arguments(dt, position_view, new_position_view, 
                            speed_view, new_speed_view,
                            masse_view, l_in):

    cdef int planets = len(position_view)

    for i in range(planets):
        l_in.append((dt, position_view, new_position_view, 
                     speed_view, new_speed_view,
                     masse_view, i))   

@cython.boundscheck(False)
@cython.cdivision(True)            
def single_step(ipyparallel_client, args_dt, position, speed, masse):              

        #Umwandlung in MemoryView
        cdef double [:, :] position_view = position
        cdef double [:, :] speed_view = speed
        cdef double [:, :] new_position_view = position
        cdef double [:, :] new_speed_view = speed 
        cdef double [:] masse_view = masse

        #Statische Variable
        cdef double dt = args_dt
        cdef int number_planets = len(position)
        cdef int i, current_planet

        cdef int thread_num = cpu_count()  
        cdef int chunksize_number=number_planets//(4*thread_num)
        cdef double [:] single_planet_view = np.zeros(6,  dtype=np.float64)         

        arguments_list = []
        single_step_arguments(dt, position_view, new_position_view, 
                            speed_view, new_speed_view,
                            masse_view, arguments_list)

        lview = ipyparallel_client.load_balanced_view()
        result_list = lview.map_sync(single_planet, arguments_list)

        for current_planet in range(number_planets):

            single_planet_view = result_list[current_planet]

            new_position_view[current_planet][0] = single_planet_view[0]
            new_position_view[current_planet][1] = single_planet_view[1]
            new_position_view[current_planet][2] = single_planet_view[2]

            new_position_view[current_planet][1] = single_planet_view[3]
            new_position_view[current_planet][2] = single_planet_view[4]
            new_position_view[current_planet][3] = single_planet_view[5]

        position_view = new_position_view
        speed_view = new_position_view

        position = np.asarray(position_view)
        speed = np.asarray(speed_view)   

[stdout:0] Overwriting ./pubyc_renderer_refactor/simulation_engine.pyx
[stdout:1] Overwriting ./pubyc_renderer_refactor/simulation_engine.pyx
[stdout:2] Overwriting ./pubyc_renderer_refactor/simulation_engine.pyx
[stdout:3] Overwriting ./pubyc_renderer_refactor/simulation_engine.pyx
[stdout:4] Overwriting ./pubyc_renderer_refactor/simulation_engine.pyx
[stdout:5] Overwriting ./pubyc_renderer_refactor/simulation_engine.pyx
[stdout:6] Overwriting ./pubyc_renderer_refactor/simulation_engine.pyx
[stdout:7] Overwriting ./pubyc_renderer_refactor/simulation_engine.pyx


In [9]:
%%px
%%file ./pubyc_renderer_refactor/setup.py
# Aufruf: python3 setup.py build_ext --inplace
# Windows: zusaetzliche Option --compiler=mingw32
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
import numpy

ext_modules=[ Extension("calculation_function", ["calculation_function.pyx"],
        extra_compile_args=['-O3'], libraries=['m']),
             Extension("simulation_engine", ["simulation_engine.pyx"],
        extra_compile_args=['-O3', '-fopenmp'], libraries=['m'],
        #extra_compile_args=['-O3'], libraries=['m'],
        extra_link_args=['-fopenmp'],
)      
        #Maybe needed
        #extra_compile_args=['-O3'], libraries=['m'],
        #include_dirs=[numpy.get_include()]),
]   
setup( name = 'cython demo',
  cmdclass = {'build_ext': build_ext},
  ext_modules = ext_modules,
     include_dirs=[numpy.get_include()])

[stdout:0] Overwriting ./pubyc_renderer_refactor/setup.py
[stdout:1] Overwriting ./pubyc_renderer_refactor/setup.py
[stdout:2] Overwriting ./pubyc_renderer_refactor/setup.py
[stdout:3] Overwriting ./pubyc_renderer_refactor/setup.py
[stdout:4] Overwriting ./pubyc_renderer_refactor/setup.py
[stdout:5] Overwriting ./pubyc_renderer_refactor/setup.py
[stdout:6] Overwriting ./pubyc_renderer_refactor/setup.py
[stdout:7] Overwriting ./pubyc_renderer_refactor/setup.py


In [15]:
%%px
%%bash

cd ./pubyc_renderer_refactor
#Compilen
python3 setup.py build_ext --inplace

[stdout:0] 
running build_ext
skipping 'calculation_function.c' Cython extension (up-to-date)
skipping 'simulation_engine.c' Cython extension (up-to-date)
[stdout:1] 
running build_ext
skipping 'calculation_function.c' Cython extension (up-to-date)
skipping 'simulation_engine.c' Cython extension (up-to-date)
[stdout:2] 
running build_ext
skipping 'calculation_function.c' Cython extension (up-to-date)
skipping 'simulation_engine.c' Cython extension (up-to-date)
[stdout:3] 
running build_ext
skipping 'calculation_function.c' Cython extension (up-to-date)
skipping 'simulation_engine.c' Cython extension (up-to-date)
[stdout:4] 
running build_ext
skipping 'calculation_function.c' Cython extension (up-to-date)
skipping 'simulation_engine.c' Cython extension (up-to-date)
[stdout:5] 
running build_ext
skipping 'calculation_function.c' Cython extension (up-to-date)
skipping 'simulation_engine.c' Cython extension (up-to-date)
[stdout:6] 
running build_ext
skipping 'calculation_function.c' Cython

In [11]:
#Sonnesystem

python_position = [
    #"Sun"
    [0, 0, 0],     
    #"Mercury"
    [57_909_175_000, 0, 0], 
    #"Venus"
    [108_208_930_000, 0, 0],    
    #"Earth"
    [149_597_890_000, 0, 0],    
    #"Moon"
    [149_597_890_000, 384_400_000, 0],    
    #"Mars"
    [227_936_640_000, 0, 0],    
    #"Jupiter"
    [778_412_020_000, 0, 0],    
    #"Saturn"
    [1_426_725_400_000, 0, 0],    
    #"Uranus"
    [2_870_972_200_000, 0, 0],    
    #"Neptune"
    [4_498_252_900_000, 0, 0]
]

python_speed = [
    #"Sun"
    [0, 0, 0],     
    #"Mercury"
    [0, 47_872, 0], 
    #"Venus"
    [0, 35_021, 0],    
    #"Earth"
    [0, 29_786, 0],    
    #"Moon"
    [-1_022, 0, 0],    
    #"Mars"
    [0, 24_131, 0],    
    #"Jupiter"
    [0, 13_069, 0],    
    #"Saturn"
    [0, 9_672, 0],    
    #"Uranus"
    [0, 6_835, 0],    
    #"Neptune"
    [0, 5_477, 0]
]

python_masse = [
    #"Sun"
    1.9889 * 10 ** 30,     
    #"Mercury"
    3.3022 * 10 ** 23, 
    #"Venus"
    4.8685 * 10 ** 24,    
    #"Earth"
    5.97219 * 10 ** 24,    
    #"Moon"
    7.34767309 * 10 ** 22,    
    #"Mars"
    6.4185 * 10 ** 23,    
    #"Jupiter"
    1.8987 * 10 ** 27,    
    #"Saturn"
    5.6851 * 10 ** 26,    
    #"Uranus"
    8.6849 * 10 ** 25,    
    #"Neptune"
    1.0244 * 10 ** 26
]

## Local Code

In [14]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append("./pubyc_renderer_refactor")

from simulation_engine import single_step
import matplotlib.pyplot as plt
import numpy as np

plot_list = []
dt = 60 * 60 * 24
year = 60 * 60 * 24 * 365 * 165
iteration = (int)(year / dt)

position = np.array(python_position, dtype=np.float64)
speed = np.array(python_speed,dtype=np.float64)
masse = np.array(python_masse,dtype=np.float64)

for i in range(iteration):
    single_step(c, dt, position, speed, masse)
    pos_list_entry = []
    for list_index in range(len(python_position)):
        pos_list_entry.append(np.array((position[list_index][0], position[list_index][1],\
        position[list_index][2]), np.float64))
        
    plot_list.append(np.array(pos_list_entry, np.float64))

ModuleNotFoundError: No module named 'simulation_engine'

In [ ]:
plt.subplot(111)
plt.plot([x[0][0] for x in plot_list], [x[0][1] for x in plot_list], label="Sun")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

plt.subplot(111)
plt.plot([x[1][0] for x in plot_list], [x[1][1] for x in plot_list], label="Mercury")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

plt.subplot(111)
plt.plot([x[2][0] for x in plot_list], [x[2][1] for x in plot_list], label="Venus")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

plt.subplot(111)
plt.plot([x[3][0] for x in plot_list], [x[3][1] for x in plot_list], label="Earth")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

plt.subplot(111)
plt.plot([x[4][0] for x in plot_list], [x[4][1] for x in plot_list], label="Moon")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

plt.subplot(111)
plt.plot([x[5][0] for x in plot_list], [x[5][1] for x in plot_list], label="Mars")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

plt.subplot(111)
plt.plot([x[6][0] for x in plot_list], [x[6][1] for x in plot_list], label="Jupiter")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

plt.subplot(111)
plt.plot([x[7][0] for x in plot_list], [x[7][1] for x in plot_list], label="Saturn")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

plt.subplot(111)
plt.plot([x[8][0] for x in plot_list], [x[8][1] for x in plot_list], label="Uranus")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

plt.subplot(111)
plt.plot([x[9][0] for x in plot_list], [x[9][1] for x in plot_list], label="Neptune")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

print("plot_list_length", len(plot_list))
for _ in range (20):
    print(plot_list[_])

## Cluster Test

In [ ]:
%%px 
%%cython

def loop(start_number, times):
    for i in range (times):
        start_number += 1
    return start_number

In [ ]:
def IPython_cluster():
    def call(start_number, times):
        return loop(start_number, times)
    
    l_in_args1 = [1,1,10,1]
    l_in_args2 = [100,100,200,2000]
    lview = c.load_balanced_view()
    lOut = lview.map_sync(call, l_in_args1, l_in_args2)
    return lOut

In [ ]:
to_print = IPython_cluster()
print(to_print)